## A. Add the dates
## B. Create long names for the peaks
Manually add names (no need to get fancy with the naming)
## To do: (Add qc?)


In [1]:
parameter_file = '../parameters/params.yaml'
import yaml
import sys
import pandas as pd
import os
from os.path import join, basename
import glob
import shutil
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.decomposition import TruncatedSVD
from scipy import sparse as sp

with open(parameter_file,'r') as f:
    doc = yaml.load(f)
    
    
samples = pd.read_csv("../parameters/samples.tsv", sep="\t", index_col=0)
samples
    
data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
#annotation=doc['annotation_gtf']
annotation=doc['annotation']

from Homer import *

os.chdir(doc["results"])

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:16: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  app.launch_new_instance()


In [2]:
samples

,Experiment,Full,Name,Tissue
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz,ATAC,CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001...,BMDM1hKLA_ATAC1,BMDM1hKLA
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz,ATAC,CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R...,BMDM1hKLA_ATAC2,BMDM1hKLA
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz,GRO,CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_...,BMDM1hKLA_GRO1,BMDM1hKLA
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz,GROCap,CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_...,BMDM1hKLA_GROCap1,BMDM1hKLA
CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_R1_001.fastq.gz,ATAC,CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_...,BMDMwt_ATAC1,BMDMwt
...,...,...,...,...
NL5_Liver_S5_L006_R1_001.fastq.gz,RNASeq,NL5_Liver_S5_L006_R1_001.fastq.gz,Liver_RNASeq1,Liver
NL6_Heart_S6_L006_R1_001.fastq.gz,RNASeq,NL6_Heart_S6_L006_R1_001.fastq.gz,Heart_RNASeq1,Heart
NL7_Brain2_S7_L006_R1_001.fastq.gz,RNASeq,NL7_Brain2_S7_L006_R1_001.fastq.gz,Brain_RNASeq2,Brain
NL8_Muscle_S8_L006_R1_001.fastq.gz,RNASeq,NL8_Muscle_S8_L006_R1_001.fastq.gz,Muscle_RNASeq1,Muscle


# Add in dates

In [19]:
data_folder = "/data/isshamie/dropbox/TSS"
dates = {"d1":"171227_hamster_GRO_mStart", "d2":"mSTART", "d3":"161115_GRO_ATAC"}


In [22]:
samples["Date"] = ''
for d in dates:
    for i in glob.glob(join(data_folder, dates[d], "*.fastq*.gz")):
        i = basename(i)
        print(i)
        if i in samples.index:
            if samples.loc[i, 'Date'] != '':
                print("sample with two dates!?", i, d)
            else:
                samples.at[i,'Date'] = d
                
samples["Date"]



CHBloondnegSpleen_GRO_JHS1032_SD_TCCCGA_S47_L003_R1_001.fastq (2).gz
CHBloondnegSpleen_GRO_JHS1032_SD_TCCCGA_S47_L003_R1_001.fastq (3).gz
CHBloondnegSpleen_GRO_JHS1032_SD_TCCCGA_S47_L003_R1_001.fastq.gz
CHBloondnegSpleen_GRO_JHS1039_SD_GTCCGC_S54_L003_R1_001.fastq.gz
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz
CHBMDMVehic1_ATAC_JHS997_SD_GTGTGGTG_S8_L001_R1_001.fastq.gz
CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_R1_001.fastq.gz
CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001.fastq.gz
CHBMDMwt_GRO_JHS1033_SD_TAGCTT_S48_L003_R1_001.fastq.gz
CHBrain7neg1_mSTART_JHS1061_SD_CACGAT_S83_L004_R1_001.fastq.gz
CHBrainnegmaybe7neg1_mSTARTinput_JHS1082_SD_CACGAT_S104_L005_R1_001.fastq.gz
CHBrain_ATAC_JHS992_SD_ACCACTGT_S4_L001_R1_001.fastq.gz
CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.fastq.gz
CHBr

CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz    d1
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz      d1
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz        d1
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz        d1
CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_R1_001.fastq.gz     d1
                                                                  ..
NL5_Liver_S5_L006_R1_001.fastq.gz                                   
NL6_Heart_S6_L006_R1_001.fastq.gz                                   
NL7_Brain2_S7_L006_R1_001.fastq.gz                                  
NL8_Muscle_S8_L006_R1_001.fastq.gz                                  
NL9_Kidney_S9_L006_R1_001.fastq.gz                                  
Name: Date, Length: 78, dtype: object

In [23]:
samples[samples["Date"] == ""]

,Experiment,Full,Name,Tissue,Date
Hamster-CHO-ATAC-SD332_S22_R1_001.fastq.gz,ATAC,Hamster-CHO-ATAC-SD332_S22_R1_001.fastq.gz,CHO_ATAC2,CHO,
Hamster-CHO-ATAC-SD335_S23_R1_001.fastq.gz,ATAC,Hamster-CHO-ATAC-SD335_S23_R1_001.fastq.gz,CHO_ATAC4,CHO,
NL10_Misc_S10_L006_R1_001.fastq.gz,RNASeq,NL10_Misc_S10_L006_R1_001.fastq.gz,Misc_RNASeq1,MiscOrgans,
NL11_Brain7-1_S11_L006_R1_001.fastq.gz,RNASeq,NL11_Brain7-1_S11_L006_R1_001.fastq.gz,Brain_RNASeq1,Brain,
NL1_Pancreas_S1_L006_R1_001.fastq.gz,RNASeq,NL1_Pancreas_S1_L006_R1_001.fastq.gz,Pancreas_RNASeq1,Pancreas,
NL2_ReproductiveTract_S2_L006_R1_001.fastq.gz,RNASeq,NL2_ReproductiveTract_S2_L006_R1_001.fastq.gz,ReproductiveTract_RNASeq1,FemaleReproductive,
NL3_Spleen_S3_L006_R1_001.fastq.gz,RNASeq,NL3_Spleen_S3_L006_R1_001.fastq.gz,Spleen_RNASeq1,Spleen,
NL4_Lung_S4_L006_R1_001.fastq.gz,RNASeq,NL4_Lung_S4_L006_R1_001.fastq.gz,Lung_RNASeq1,Lung,
NL5_Liver_S5_L006_R1_001.fastq.gz,RNASeq,NL5_Liver_S5_L006_R1_001.fastq.gz,Liver_RNASeq1,Liver,
NL6_Heart_S6_L006_R1_001.fastq.gz,RNASeq,NL6_Heart_S6_L006_R1_001.fastq.gz,Heart_RNASeq1,Heart,


## Create long names for peaks (csRNA only have different names)